In [1]:
import numpy as np
from gridworld import GridWorld

import matplotlib.pyplot as plt

import irl.linear_irl as linear_irl
import irl.mdp.gridworld as gridworld

In [7]:
def print_policy(policy_matrix):
    '''Print the policy using specific symbol.

    * terminal state
    ^ > v < up, right, down, left
    # obstacle
    '''
    counter = 0
    shape = policy_matrix.shape
    policy_string = ""
    for row in range(shape[0]):
        for col in range(shape[1]):
            if(policy_matrix[row,col] == -1): policy_string += " *  "
            elif(policy_matrix[row,col] == 0): policy_string += " ^  "
            elif(policy_matrix[row,col] == 1): policy_string += " >  "
            elif(policy_matrix[row,col] == 2): policy_string += " v  "
            elif(policy_matrix[row,col] == 3): policy_string += " <  "
            elif(np.isnan(policy_matrix[row,col])): policy_string += " #  "
            counter += 1
        policy_string += '\n'
    print(policy_string)

def get_return(state_list, gamma):
    '''Get the return for a list of action-state values.

    @return get the Return
    '''
    counter = 0
    return_value = 0
    for visit in state_list:
        reward = visit[2]
        return_value += reward * np.power(gamma, counter)
        counter += 1
    return return_value

def update_policy(episode_list, policy_matrix, state_action_matrix):
    '''Update a policy making it greedy in respect of the state-action matrix.

    @return the updated policy
    '''
    for visit in episode_list:
        observation = visit[0]
        col = observation[1] + (observation[0]*4)
        if(policy_matrix[observation[0], observation[1]] != -1):
            policy_matrix[observation[0], observation[1]] = \
                np.argmax(state_action_matrix[:,col])
    return policy_matrix

In [33]:
env = GridWorld(5, 5)

#Define the state matrix
state_matrix = np.zeros((5,5))
state_matrix[0, 4] = 1
print("State Matrix:")
print(state_matrix)

#Define the reward matrix
reward_matrix = np.full((5,5), 0)
reward_matrix[0, 4] = 1
print("Reward Matrix:")
print(reward_matrix)

#Define the transition matrix
transition_matrix = np.array([[0.7, 0.1, 0.1, 0.1],
                              [0.1, 0.7, 0.1, 0.1],
                              [0.1, 0.1, 0.7, 0.1],
                              [0.1, 0.1, 0.1, 0.7]])

#Random policy
policy_matrix = np.random.randint(low=0, high=4, size=(5, 5)).astype(np.float32)
policy_matrix[0,4] = -1

#Set the matrices in the world
env.setStateMatrix(state_matrix)
env.setRewardMatrix(reward_matrix)
env.setTransitionMatrix(transition_matrix)

state_action_matrix = np.random.random_sample((4,25)) # Q
#init with 1.0e-10 to avoid division by zero
running_mean_matrix = np.full((4,25), 1.0e-10)
gamma = 0.999
tot_epoch = 30
print_epoch = 5

for epoch in range(tot_epoch):
    #Starting a new episode
    episode_list = list()
    #Reset and return the first observation and reward
    observation = env.reset(exploring_starts=False)

    is_starting = True
    for _ in range(1000):
        #Take the action from the action matrix
        action = policy_matrix[observation[0], observation[1]]
        #If the episode just started then it is
            #necessary to choose a random action (exploring starts)
        if(is_starting):
            action = np.random.randint(0, 4)
            is_starting = False
        #Move one step in the environment and get obs and reward
        new_observation, reward, done = env.step(action)
        #Append the visit in the episode list
        episode_list.append((observation, action, reward))
        observation = new_observation
        
        if done: break
            
        #The episode is finished, now estimating the utilities
        counter = 0
        #Checkup to identify if it is the first visit to a state
        checkup_matrix = np.zeros((4,25))
        #This cycle is the implementation of First-Visit MC.

        for visit in episode_list:
            observation = visit[0]
            action = visit[1]
            col = int(observation[1] + (observation[0]*4))
            row = int(action)
            if(checkup_matrix[row, col] == 0):
                return_value = get_return(episode_list[counter:], gamma)
                running_mean_matrix[row, col] += 1
                state_action_matrix[row, col] += return_value
                checkup_matrix[row, col] = 1
            counter += 1
        #Policy Update
        policy_matrix = update_policy(episode_list,
                                      policy_matrix,
                                      state_action_matrix/running_mean_matrix)

    if(epoch % print_epoch == 0):
        print("------>",epoch % print_epoch)
        print("State-Action matrix after " + str(epoch+1) + " iterations:")
        print(state_action_matrix / running_mean_matrix)
        print("Policy matrix after " + str(epoch+1) + " iterations:")
        print(policy_matrix)
        print("Policy Directions")
        print_policy(policy_matrix)


#Time to check the utility matrix obtained
print("Utility matrix after " + str(tot_epoch) + " iterations:")
print(state_action_matrix / running_mean_matrix)
        
print("-------END--------")

State Matrix:
[[0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
Reward Matrix:
[[0 0 0 0 1]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
('------>', 0)
State-Action matrix after 1 iterations:
[[9.04915711e+09 9.14203588e+09 5.98113494e+09 4.51999721e+09
  5.66044743e+08 4.57367081e+09 4.95383213e+09 8.44101313e+09
  1.51467132e+09 9.25609627e+09 5.40014766e+09 3.56257496e+09
  6.95767866e+09 8.64774793e+09 5.49980032e+09 7.52974974e+09
  1.57975385e-03 2.46520549e+09 6.51065756e+09 4.39105621e+09
  1.95427167e+09 8.89142644e+09 6.00379767e+09 8.17092020e+09
  6.87226173e+09]
 [2.17815216e+09 5.39106403e+08 5.57379179e+09 4.19195681e+08
  1.79086699e+09 4.29874164e+09 2.06881745e+09 3.09632514e+09
  1.53235871e+09 1.49048905e+09 5.54435287e+09 2.52223182e+09
  9.69278114e+08 8.26856548e+09 3.31934266e+09 4.30623501e+09
  5.12712210e-04 4.08136625e+09 6.25517450e+09 7.44378437e+09
  3.56653375e+09 3.18238321e+09 7.85115510e+09 5.08855972e+

('------>', 0)
State-Action matrix after 26 iterations:
[[9.04915711e+09 9.14203588e+09 5.98113494e+09 4.51999721e+09
  5.66044743e+08 4.57367081e+09 4.95383213e+09 8.44101313e+09
  1.51467132e+09 9.25609627e+09 5.40014766e+09 3.56257496e+09
  6.95767866e+09 8.64774793e+09 5.49980032e+09 7.52974974e+09
  3.92934779e-05 2.46520549e+09 6.51065756e+09 4.39105621e+09
  1.95427167e+09 8.89142644e+09 6.00379767e+09 8.17092020e+09
  6.87226173e+09]
 [2.17815216e+09 5.39106403e+08 5.57379179e+09 4.19195681e+08
  1.79086699e+09 4.29874164e+09 2.06881745e+09 3.09632514e+09
  1.53235871e+09 1.49048905e+09 5.54435287e+09 2.52223182e+09
  9.69278114e+08 8.26856548e+09 3.31934266e+09 4.30623501e+09
  3.73027988e-05 4.08136625e+09 6.25517450e+09 7.44378437e+09
  3.56653375e+09 3.18238321e+09 7.85115510e+09 5.08855972e+09
  7.53051314e+09]
 [4.01408164e+08 4.63274944e+09 8.38985508e+09 3.50439033e+09
  4.22529725e+09 9.59793557e+09 5.32770804e+09 7.32914127e+09
  1.02555309e+09 9.60688700e+08 8.127173

In [39]:
state_value_matrix = state_action_matrix.max(axis=0)
state_value_matrix

array([0.90491571, 0.91420359, 0.83898551, 0.58878459, 0.55719487,
       0.98332775, 0.85434505, 0.84410131, 0.15323587, 0.92560963,
       0.87032402, 0.92411873, 0.69576787, 0.86477479, 0.88329659,
       0.75297497, 0.93636278, 0.57862701, 0.93284128, 0.74437844,
       0.93540401, 0.88914264, 0.78511551, 0.81709202, 0.75305131])

In [ ]:
from cvxopt import matrix, solvers

def irl(n_states, n_actions, transition_probability, policy, discount, Rmax,
        l1):
    """
    Find a reward function with inverse RL as described in Ng & Russell, 2000.

    n_states: Number of states. int.
    n_actions: Number of actions. int.
    transition_probability: NumPy array mapping (state_i, action, state_k) to
        the probability of transitioning from state_i to state_k under action.
        Shape (N, A, N).
    policy: Vector mapping state ints to action ints. Shape (N,).
    discount: Discount factor. float.
    Rmax: Maximum reward. float.
    l1: l1 regularisation. float.
    -> Reward vector
    """

    A = set(range(n_actions))  # Set of actions to help manage reordering
                               # actions.
    # The transition policy convention is different here to the rest of the code
    # for legacy reasons; here, we reorder axes to fix this. We expect the
    # new probabilities to be of the shape (A, N, N).
    transition_probability = np.transpose(transition_probability, (1, 0, 2))

    def T(a, s):
        """
        Shorthand for a dot product used a lot in the LP formulation.
        """

        return np.dot(transition_probability[policy[s], s] -
                      transition_probability[a, s],
                      np.linalg.inv(np.eye(n_states) -
                        discount*transition_probability[policy[s]]))

    # This entire function just computes the block matrices used for the LP
    # formulation of IRL.

    # Minimise c . x.
    c = -np.hstack([np.zeros(n_states), np.ones(n_states),
                    -l1*np.ones(n_states)])
    zero_stack1 = np.zeros((n_states*(n_actions-1), n_states))
    T_stack = np.vstack([
        -T(a, s)
        for s in range(n_states)
        for a in A - {policy[s]}
    ])
    I_stack1 = np.vstack([
        np.eye(1, n_states, s)
        for s in range(n_states)
        for a in A - {policy[s]}
    ])
    I_stack2 = np.eye(n_states)
    zero_stack2 = np.zeros((n_states, n_states))

    D_left = np.vstack([T_stack, T_stack, -I_stack2, I_stack2])
    D_middle = np.vstack([I_stack1, zero_stack1, zero_stack2, zero_stack2])
    D_right = np.vstack([zero_stack1, zero_stack1, -I_stack2, -I_stack2])

    D = np.hstack([D_left, D_middle, D_right])
    b = np.zeros((n_states*(n_actions-1)*2 + 2*n_states, 1))
    bounds = np.array([(None, None)]*2*n_states + [(-Rmax, Rmax)]*n_states)

    # We still need to bound R. To do this, we just add
    # -I R <= Rmax 1
    # I R <= Rmax 1
    # So to D we need to add -I and I, and to b we need to add Rmax 1 and Rmax 1
    D_bounds = np.hstack([
        np.vstack([
            -np.eye(n_states),
            np.eye(n_states)]),
        np.vstack([
            np.zeros((n_states, n_states)),
            np.zeros((n_states, n_states))]),
        np.vstack([
            np.zeros((n_states, n_states)),
            np.zeros((n_states, n_states))])])
    b_bounds = np.vstack([Rmax*np.ones((n_states, 1))]*2)
    D = np.vstack((D, D_bounds))
    b = np.vstack((b, b_bounds))
    A_ub = matrix(D)
    b = matrix(b)
    c = matrix(c)
    results = solvers.lp(c, A_ub, b)
    r = np.asarray(results["x"][:n_states], dtype=np.double)

    return r.reshape((n_states,))

## Random State Transition Matrix

In [42]:
random_state_transition_matrix = np.random.rand(25,25)
random_state_transition_matrix = random_state_transition_matrix/ \
                                random_state_transition_matrix.sum(axis=1)[:,None]
random_state_transition_matrix.shape

(25, 25)

## With a handcrafted State Transition Matrix

In [ ]:
grid_size = 5
discount = 0.2
"""
Run linear programming inverse reinforcement learning on the gridworld MDP.

Plots the reward function.

grid_size: Grid size. int.
discount: MDP discount factor. float.
"""

wind = 0.3
trajectory_length = 3*grid_size

gw = gridworld.Gridworld(grid_size, wind, discount)

ground_r = np.array([gw.reward(s) for s in range(gw.n_states)])

#policy = [gw.optimal_policy_deterministic(s) for s in range(gw.n_states)]
r = linear_irl.irl(gw.n_states, gw.n_actions, gw.transition_probability,
        policy, gw.discount, 1, 5)

print(r.shape)
print(gw.optimal_policy_deterministic)
print(np.array(policy).reshape(5,5))

plt.subplot(1, 2, 1)
plt.pcolor(ground_r.reshape((grid_size, grid_size)))
plt.colorbar()
plt.title("Groundtruth reward")
plt.subplot(1, 2, 2)
plt.pcolor(r.reshape((grid_size, grid_size)))
plt.colorbar()
plt.title("Recovered reward")
plt.show()

